In [71]:
%serialconnect

serial exception on close write failed: [Errno 5] Input/output error
Connecting to --port=/dev/ttyUSB2 --baud=115200 
Ready.


In [42]:
%sendtofile --source UDPblackviewphone.py


Sent 30 lines (819 bytes) to UDPblackviewphone.py.


In [15]:
%sendtofile --source UDPblackviewphone.py
%sendtofile --source ubxminimal.py

Sent 66 lines (2004 bytes) to ubxminimal.py.


In [42]:
%sendtofile deviceletter.txt
C

Sent 1 lines (1 bytes) to deviceletter.txt.


In [14]:
%serialconnect

serial exception on close write failed: [Errno 5] Input/output error
Connecting to --port=/dev/ttyUSB1 --baud=115200 
Traceback (most recent call last):
  File "main.py", line 25, in <module>
  File "UDPblackviewphone.py", line 12, in connectActivePhone
KeyboardInterrupt: 
MicroPython v1.10 on 2019-01-25; ESP32 module with ESP32
Type "help()" for more information.
>>> 
>>> 
raw REPL; CTRL-B to exit
>Ready.


In [16]:
%sendtofile hotspots.txt

BV6000         beckaaaa    192.168.43.1   9042
ES_3041        43900000    192.168.43.1   9042
AndroidAP      napz5037    192.168.43.1   9042
DoESLiverpool  decafbad00  node-red.dynamicdevices.co.uk   4006


Sent 4 lines (205 bytes) to hotspots.txt.


In [17]:
%sendtofile main.py

import time, machine

# delay long enough for Ctrl-C before the wlan bricks it for debugging
# reset_cause=1:PWRON_RESET, 2:HARD, 5:SOFT (brownout detected)
pled = machine.Pin(2, machine.Pin.OUT)
for i in range(2*(machine.reset_cause())):
    pled.value(1-(i%2))
    time.sleep_ms(400)  

# Set up flashing LED timer (better control than PWM)
pgled = machine.Pin(23, machine.Pin.OUT)
timeracc = 0
timermax = 1000
timerlight = 100
timeradd = 50
def timercallback(t):
    global timeracc
    timeracc = (timeracc + timeradd)%timermax
    pgled.value(int(timeracc<timerlight))
timer = machine.Timer(-1)
timer.init(period=50, mode=machine.Timer.PERIODIC, callback=timercallback)

# Serial connection to the UBlox GPS device
from ubxminimal import uartUBX, initUBX
initUBX()

for i in range(11):
    pled.value(i%2)
    time.sleep_ms(200)

print("RESET_CAUSE", machine.reset_cause())
deviceletter = open("deviceletter.txt", "r").read()
print("deviceletter", deviceletter)

from UDPblackviewphone import connectActivePhone
while True:
    androidipnumber, portnumber = connectActivePhone(pled)
    if androidipnumber is not None:
        break
    for j in range(3):
        for i in range(7):
            pled.value(i%2)
            time.sleep_ms(100)
        time.sleep_ms(800)

for i in range(21):
    pled.value(i%2)
    time.sleep_ms(80)

import socket, uselect

ubxbuffer = bytearray(1000)
mubxbuffer = memoryview(ubxbuffer)
timelastledsignal = 0
obj = None
while True:
    timermax = 4000  # slow down signal to show it's broken
    try:
        ss = socket.socket()
        ss.settimeout(1)
        print(ss)
        ss.connect(socket.getaddrinfo(androidipnumber, portnumber)[0][-1])
        s = ss.makefile('rwb', 0)
        print(s.readline())
        s.write(b"%c%c%c%c"%(deviceletter,deviceletter,deviceletter,deviceletter))
        poller = uselect.poll()
        poller.register(uartUBX, uselect.POLLIN)
        poller.register(s, uselect.POLLIN)
        while True:
            for obj, evt in poller.ipoll(100):
                if obj == uartUBX and evt == uselect.POLLIN:
                    n = uartUBX.readinto(ubxbuffer)
                    if n is not None:
                        s.write(mubxbuffer[:n])
                        pled.value(1-pled.value())
                    if time.ticks_ms() - timelastledsignal > 2000:
                        timermax = 3000  # slow down signal to show it's broken

                if obj == s and evt == uselect.POLLIN:
                    timelastledsignal = time.ticks_ms()
                    timermax = 1000
                    l = s.readline()
                    try:
                        print(l)
                        timeradd, timerlight = list(map(int, l.split()))
                    except ValueError as e:
                        print("ValueError", e)
    except OSError as e:
        print("OSError", e, obj)
        time.sleep_ms(2000)



Sent 91 lines (2913 bytes) to main.py.


In [47]:
%serialconnect

serial exception on close write failed: [Errno 5] Input/output error
Connecting to --port=/dev/ttyUSB0 --baud=115200 
Ready.


In [66]:
from ubxminimal import uartUBX, initUBX


In [67]:
print(uartUBX.read(100))

b'\xc107,,66,26,067,14,67,09,117,,73,44,309,*60\r\n$GLGSV,3,2,10,74,08,339,,79,04,178,,80,45,212,,81,25,061'


In [68]:
import uselect
pl = uselect.poll()

In [71]:
pl.register(uartUBX, uselect.POLLIN)

In [72]:
help(pl)

object <poll> is of type poll
  register -- <function>
  unregister -- <function>
  modify -- <function>
  poll -- <function>
  ipoll -- <function>


In [73]:
print(pl.poll(10))

[(UART(1, baudrate=9600, bits=8, parity=None, stop=1, tx=17, rx=16, rts=-1, cts=-1, txbuf=256, rxbuf=256, timeout=0, timeout_char=2), 1)]


In [70]:
help(uselect)

object <module 'uselect'> is of type module
  __name__ -- uselect
  select -- <function>
  poll -- <function>
  POLLIN -- 1
  POLLOUT -- 4
  POLLERR -- 8
  POLLHUP -- 16


In [64]:
import socket
s = socket.socket()
help(s)

object <socket> is of type socket
  __del__ -- <function>
  close -- <function>
  bind -- <function>
  listen -- <function>
  accept -- <function>
  connect -- <function>
  send -- <function>
  sendall -- <function>
  sendto -- <function>
  recv -- <function>
  recvfrom -- <function>
  setsockopt -- <function>
  settimeout -- <function>
  setblocking -- <function>
  makefile -- <function>
  fileno -- <function>
  read -- <function>
  readinto -- <function>
  readline -- <function>
  write -- <function>


In [2]:
%serialconnect

Connecting to --port=/dev/ttyUSB1 --baud=115200 
Traceback (most recent call last):
  File "main.py", line 29, in <module>
  File "UDPblackviewphone.py", line 12, in connectActivePhone
KeyboardInterrupt: 
MicroPython v1.10 on 2019-01-25; ESP32 module with ESP32
Type "help()" for more information.
>>> 
>>> 
raw REPL; CTRL-B to exit
>Ready.


In [3]:
del pgled
p = machine.Pin(23, machine.Pin.OUT)
p.value(1)

In [ ]:
p = machine.PWM(machine.Pin(23, machine.Pin.OUT))
p.freq(1)
p.duty(300)


In [4]:
help(machine.Timer)

object <class 'Timer'> is of type type
  __del__ -- <function>
  deinit -- <function>
  init -- <function>
  value -- <function>
  ONE_SHOT -- 0
  PERIODIC -- 1


In [36]:
timeracc = 0
timermax = 1000
timerlight = 500
timeradd = 1
def timercallback(t):
    global timeracc
    timeracc = (timeracc + timeradd)%timermax
    p.value(int(timeracc<timerlight))



In [37]:
timer.init(period=50, mode=machine.Timer.PERIODIC, callback=timercallback)

In [48]:
timeradd = 505
print(timeracc)

994


In [33]:
print(timer.value())

398


In [34]:
# https://github.com/micropython/micropython-esp32/blob/2f4dac5f121a59fc187c1d9c1f9eade365b3aba1/py/mperrno.h
# OSERROR ENFILE = 23 # File table overflow
# OSERROR 118 (don't know)


object <class 'Timer'> is of type type
  __del__ -- <function>
  deinit -- <function>
  init -- <function>
  value -- <function>
  ONE_SHOT -- 0
  PERIODIC -- 1


In [88]:
%disconnect

attempt to exit paste mode
[\r\x03\x02] b'\r\nMicroPython v1.10 on 2019-01-25; ESP32 module with ESP32\r\nType "help()" for more information.\r\n>>> '
Closing serial Serial<id=0x7f5def4951d0, open=True>(port='/dev/ttyUSB0', baudrate=115200, bytesize=8, parity='N', stopbits=1, timeout=0.5, xonxoff=False, rtscts=False, dsrdtr=False)


In [92]:
%serialconnect

Connecting to --port=/dev/ttyUSB0 --baud=115200 
Ready.


In [93]:
deviceletter = open("deviceletter.txt", "r").read()
print(b"%c%c%c%c"%(deviceletter,deviceletter,deviceletter,deviceletter))

b'AAAA'


In [100]:
#print(s.readline())
print(pgled)
pgled.freq(2)
pgled.duty(700)

PWM(23, freq=10, duty=700)


In [94]:
for i in range(20):
    print(s.readline())

Traceback (most recent call last):
  File "<stdin>", line 2, in <module>
OSError: [Errno 110] ETIMEDOUT


In [116]:
#s.settimeout(0)
s.write("hitehrere")
s.settimeout(0)
k = s.readline()
print(k)

None


In [1]:
%serialconnect

Connecting to --port=/dev/ttyUSB3 --baud=115200 
Ready.


In [3]:
# for testing UBX is wired in
import machine
uartUBX = machine.UART(1, baudrate=9600, rx=16, tx=17)
print(uartUBX.read())

None


In [4]:
for l in uartUBX:
    print(l)

b'$GNRMC,,V,,,,,,,,,,N*4D\r\n'
b'$GNVTG,,,,,,,,,N*2E\r\n'
b'$GNGGA,,,,,,0,00,99.99,,,,,,*56\r\n'
b'$GNGSA,A,1,,,,,,,,,,,,,99.99,99.99,99.99*2E\r\n'
b'$GNGSA,A,1,,,,,,,,,,,,,99.99,99.99,99.99*2E\r\n'
b'$GPGSV,1,1,00*79\r\n'
b'$GLGSV,1,1,00*65\r\n'
b'$GNGLL,,,,,,V,N*7A\r\n'
b'$GNZDA,,,,,00,00*56\r\n'
b'$GNRMC,,V,,,,,,,,,,N*4D\r\n'
b'$GNVTG,,,,,,,,,N*2E\r\n'
b'$GNGGA,,,,,,0,00,99.99,,,,,,*56\r\n'
b'$GNGSA,A,1,,,,,,,,,,,,,99.99,99.99,99.99*,1,,,,,,,,,,,,,99.99,99.99,99.99*2E\r\n'
b'$GPGSV,1,1,00*79\r\n'
b'$GLGSV,1,1,00*65\r\n'
b'$GNGLL,,,,,,V,N*7A\r\n'
b'$GNZDA,,,,,00,00*56\r\n'


In [ ]:
%sendtofile main.py

import time, machine, urandom

# delay long enough to interrupt if we need to, 
# before the uart or wlan bricks it for debugging
pled = machine.Pin(2, machine.Pin.OUT)
for i in range(11):
    pled.value(i%2)
    time.sleep_ms(500)  

from ubxminimal import uartUBX, initUBX
initUBX()

for i in range(11):
    pled.value(i%2)
    time.sleep_ms(200)

deviceletter = open("deviceletter.txt", "r").read()
port = {"A":9020, "B":9021, "C":9022}[deviceletter]
udpaddr = ("192.168.43.1", port)  # ip default for android

from UDPblackviewphone import connectBlackview, connectS5
if deviceletter == 'C':
    connectS5(pled)
else:
    connectBlackview(pled)

for i in range(11):
    pled.value(i%2)
    time.sleep_ms(100)


import socket
s = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
s.settimeout(0.005)
    
ubxbuffer = bytearray(1000)

# main loop
PCudpaddr = None
reccount = 0
while True:
    n = uartUBX.readinto(ubxbuffer)
    if n is not None:
        try:
            s.sendto(mubxbuffer[:n+2], udpaddr)
            if PCudpaddr is not None:
                s.sendto(mubxbuffer[:n+2], PCudpaddr)
        except OSError as e:
            print("dwrite", e)
        #print(x[:5])
        pled.value(1-pled.value())
    else:
        try:
            mess, PCudpaddr = s.recvfrom(50)
            print(mess, PCudpaddr)
        except OSError as e:
            pass # timeout

